## Projet 5 : Segmentez des clients d'un site e-commerce

## Contexte

Vous êtes consultant pour Olist, une entreprise brésilienne qui propose une solution de vente sur les marketplaces en ligne.

Votre rôle est d’accompagner Olist dans leur projet de monter une équipe Data et leur premier cas d’usage Data Science autour de la segmentation client.

Première urgence : l'implémentation de requêtes SQL urgentes.

Deuxième urgence :  fournir aux équipes d'e-commerce une segmentation des clients qu’elles pourront utiliser au quotidien pour leurs campagnes de communication.

Vous devrez fournir à l’équipe Marketing une description actionable de votre segmentation et de sa logique sous-jacente pour une utilisation optimale, ainsi qu’une proposition de contrat de maintenance basée sur une analyse de la stabilité des segments au cours du temps.

## Librairies utilisées avec le langage Python:

In [1]:
# Importation des librairies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import sqlite3
# from IPython.display import Image
# from sklearn.preprocessing import LabelEncoder # sklearn preprocessing for dealing with categorical variables
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
# from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import adjusted_rand_score
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.model_selection import train_test_split
import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
 # Fonction permettant de visualiser le pourcentage de valeurs manquantes par variables

def show_miss_val(data):
    variables = []
    miss_val = []

    for item in data.columns:
        variables.append(item)
        miss_val.append(data[item].isna().mean()) #Calcul du pourcentage de valeurs manquantes

    # Création d'un dataframe avec le résultat
    output = pd.DataFrame({
        'variable': variables,
        'miss_val': miss_val
    })
    # Affichage d'un message récapitulant le nombre de variables et le nombre d'entre elles qui a des valeurs manquantes
    print ("Notre dataframe a " + str(data.shape[1]) + " variables.\n Il y a " + str(output[output['miss_val']!=0].shape[0]) +" variables contenant des valeurs manquantes.")

    # Retourne les colonnes qui ont des valeurs manquantes avec le pourcentage de valeurs manquantes de chacune d'entre elles
    return output[output['miss_val']!=0].sort_values('miss_val')

In [3]:
def pd_to_sqlDB(input_df: pd.DataFrame,
                table_name: str,
                db_name: str = 'default.db') -> None:

    '''Take a Pandas dataframe `input_df` and upload it to `table_name` SQLITE table
    Args:
        input_df (pd.DataFrame): Dataframe containing data to upload to SQLITE
        table_name (str): Name of the SQLITE table to upload to
        db_name (str, optional): Name of the SQLITE Database in which the table is created.
                                 Defaults to 'default.db'.
    '''

    # Step 1: Setup local logging
    import logging
    logging.basicConfig(level=logging.INFO,
                        format='%(asctime)s %(levelname)s: %(message)s',
                        datefmt='%Y-%m-%d %H:%M:%S')

    # Step 2: Find columns in the dataframe
    cols = input_df.columns
    cols_string = ','.join(cols)
    val_wildcard_string = ','.join(['?'] * len(cols))

    # Step 3: Connect to a DB file if it exists, else crete a new file
    con = sqlite3.connect(db_name)
    cur = con.cursor()
    logging.info(f'SQL DB {db_name} created')

    # Step 4: Create Table
    sql_string = f"""CREATE TABLE {table_name} ({cols_string});"""
    cur.execute(sql_string)
    logging.info(f'SQL Table {table_name} created with {len(cols)} columns')

    # Step 5: Upload the dataframe
    rows_to_upload = input_df.to_dict(orient='split')['data']
    sql_string = f"""INSERT INTO {table_name} ({cols_string}) VALUES ({val_wildcard_string});"""
    cur.executemany(sql_string, rows_to_upload)
    logging.info(f'{len(rows_to_upload)} rows uploaded to {table_name}')

    # Step 6: Commit the changes and close the connection
    con.commit()
    con.close()

In [4]:
def sql_query_to_pd(sql_query_string: str, db_name: str ='default.db') -> pd.DataFrame:
    '''Execute an SQL query and return the results as a pandas dataframe
    Args:
        sql_query_string (str): SQL query string to execute
        db_name (str, optional): Name of the SQLITE Database to execute the query in.
                                 Defaults to 'default.db'.
    Returns:
        pd.DataFrame: Results of the SQL query in a pandas dataframe
    '''
    # Step 1: Connect to the SQL DB
    con = sqlite3.connect(db_name)

    # Step 2: Execute the SQL query
    cursor = con.execute(sql_query_string)

    # Step 3: Fetch the data and column names
    result_data = cursor.fetchall()
    cols = [description[0] for description in cursor.description]

    # Step 4: Close the connection
    con.close()

    # Step 5: Return as a dataframe
    return pd.DataFrame(result_data, columns=cols)

## Simulation pour élaborer un contrat de maintenance :


Dans le but d'établir un contrat de maintenance de l'algorithme de segmentation client, nous devons tester sa stabilité dans le temps et voir, par exemple, à quel moment les clients changent de Cluster.

Pour cela, nous devons recalculer toutes les features mois par mois.

### Création du dataframe complet :

In [5]:
data_fe_client=pd.read_csv('D:/tutorial-env/OCR/Projet5/data_fe_client.csv')
data_rfm_client=pd.read_csv('D:/tutorial-env/OCR/Projet5/data_rfm_client.csv')

In [6]:
data=data_fe_client.merge(data_rfm_client, on='customer_unique_id', how='left')
cols_to_drop = ["customer_unique_id","order_id","payment_total"]
data = data.drop(cols_to_drop, axis=1)
data = data.rename(columns={"price": "prix",
                            "montant": "montant_total",
                            "freight_value": "frais_livraison",
                            "review_score_moy": "satisfaction",
                            "estimated_delivery_delay":"date_livraison_estimee",
                            "true_delivery_delay":"date_livraison_reelle",
                            "late_delivery": "retard_livraison",
                            "payment_payment_installments": "nb_versement"})
data = data.sort_values('recence', ascending=False)
data = data.reset_index()
data

,index,date_livraison_estimee,date_livraison_reelle,retard_livraison,prix,frais_livraison,nb_versement,satisfaction,Frequence,recence,montant_total
0,69097,46.0,NaN,NaN,32.90,31.67,1.0,1.0,1,773.0,136.23
1,27245,53.0,NaN,NaN,59.50,15.56,3.0,1.0,1,772.0,75.06
2,238,17.0,NaN,NaN,0.00,0.00,2.0,1.0,1,764.0,40.95
3,49312,19.0,55.0,36.0,44.99,2.83,NaN,1.0,1,762.0,NaN
4,5521,23.0,NaN,NaN,100.00,9.34,1.0,1.0,1,745.0,109.34
...,...,...,...,...,...,...,...,...,...,...,...
96091,58524,16.0,NaN,NaN,0.00,0.00,1.0,1.0,1,18.0,137.03
96092,37382,21.5,6.0,-15.0,65.00,15.38,1.5,2.0,2,16.0,160.76
96093,65954,12.0,7.0,-5.0,178.00,19.55,1.0,1.0,3,14.0,592.65
96094,14363,31.0,15.0,-20.0,197.50,24.53,1.0,5.0,2,1.0,444.06


In [7]:
#  Imputation et normalisation du dataframe
# Créer une instance de SimpleImputer pour imputer les valeurs manquantes
imputer = SimpleImputer(strategy='median')
# Remodeler la colonne en un tableau 2D
data_temp = data['nb_versement'].values.reshape(-1, 1)
data_temp_ = data['montant_total'].values.reshape(-1, 1)
data_temp__ = data['satisfaction'].values.reshape(-1, 1)
data_temp___ = data['date_livraison_reelle'].values.reshape(-1, 1)
data_temp____ = data['retard_livraison'].values.reshape(-1, 1)

data['nb_versement'] = imputer.fit_transform(data_temp)
data['montant_total'] = imputer.fit_transform(data_temp_)
data['satisfaction'] = imputer.fit_transform(data_temp__)
data['date_livraison_reelle'] = imputer.fit_transform(data_temp___)
data['retard_livraison'] = imputer.fit_transform(data_temp____)

data_vm=show_miss_val(data)
data_vm.head(26)

Notre dataframe a 11 variables.
 Il y a 0 variables contenant des valeurs manquantes.


,variable,miss_val


### Transformation de variables pour le clustering :

In [8]:
# Transformation en racine carré de certaines variables pour le clustering
data_scale=data.copy()

data_scale['Frequence']= np.sqrt(data['Frequence'])
data_scale['montant_total']= np.sqrt(data['montant_total'])
data_scale['frais_livraison']= np.sqrt(data['frais_livraison'])
data_scale['satisfaction']= data['satisfaction']
data_scale['retard_livraison']= data['retard_livraison']
data_scale['nb_versement']= np.sqrt(data['nb_versement'])


### Découpage du dataframe par période :

In [30]:
 # Fonction permettant de decouper un dataframe par pas de période selon la variable recence en ne gardant que les individus du premier découpage
def split_df_period(df,period_init,period_step,split_nb):
    dataframes = {}
    df_temp = pd.DataFrame()
    # Découpage initial
    df_temp=df.loc[df['recence'] > period_init]
    dataframes["df_0"] = df_temp
    print ("Dataframe 0 créé. " )

    # Autres découpages par pas de période
    for i in range (1,split_nb):
      df_temp = pd.DataFrame()
      df_temp=df.loc[df['recence'] > period_init-(i*period_step)]

      df_name = "df_" + str(i)
      dataframes[df_name] = df_temp
      print ("Dataframe " + str(df_name) + " créé. " )
    return dataframes
    # Retourne un dictionnaire de dataframes découpés par période

In [31]:
df=split_df_period(data_scale,289,15,21)
df

Dataframe 0 créé. 
Dataframe df_1 créé. 
Dataframe df_2 créé. 
Dataframe df_3 créé. 
Dataframe df_4 créé. 
Dataframe df_5 créé. 
Dataframe df_6 créé. 
Dataframe df_7 créé. 
Dataframe df_8 créé. 
Dataframe df_9 créé. 
Dataframe df_10 créé. 
Dataframe df_11 créé. 
Dataframe df_12 créé. 
Dataframe df_13 créé. 
Dataframe df_14 créé. 
Dataframe df_15 créé. 
Dataframe df_16 créé. 
Dataframe df_17 créé. 
Dataframe df_18 créé. 
Dataframe df_19 créé. 
Dataframe df_20 créé. 


{'df_0':        index  date_livraison_estimee  date_livraison_reelle  retard_livraison  \
 0      69097                    46.0                   10.0             -12.0   
 1      27245                    53.0                   10.0             -12.0   
 2        238                    17.0                   10.0             -12.0   
 3      49312                    19.0                   55.0              36.0   
 4       5521                    23.0                   10.0             -12.0   
 ...      ...                     ...                    ...               ...   
 43342   4762                    24.0                   10.0             -14.0   
 43343  63967                    24.0                   19.0              -5.0   
 43344  11010                    30.0                   24.0              -6.0   
 43345  43799                    36.0                   16.0             -20.0   
 43346    232                    31.0                   11.0             -20.0   
 
      

### Clustering des dataframes découpés :

In [44]:
 # Fonction permettant de normaliser et effectuer le clustering sur un dictionnaire de dataframes
def cluster_dict_df(dict_df,cluster_cols):
    taille_dictionnaire = len(dict_df)
    liste_valeurs = list(dict_df.values())
    liste_labels = list()
    scaler = MinMaxScaler()
    for i in range (0,taille_dictionnaire):
      valeur = liste_valeurs[i]
      df_cluster = valeur[cluster_cols]

      df_norm = pd.DataFrame(scaler.fit_transform(df_cluster),columns=df_cluster.columns)
      kmeans = KMeans(init='k-means++',n_clusters=4,n_init=10,max_iter=300, random_state=42)
      kmeans.fit(df_norm)
      liste_labels.append( kmeans.labels_)
    return liste_labels
    # Retourne la liste des labels des clustering  du dictionnaire de dataframes

In [45]:
label_list=cluster_dict_df(df,['Frequence','montant_total','frais_livraison','satisfaction','retard_livraison','nb_versement'])
label_list

[array([1, 1, 1, ..., 0, 0, 3]),
 array([2, 2, 2, ..., 0, 1, 1]),
 array([2, 2, 2, ..., 1, 2, 1]),
 array([0, 0, 0, ..., 2, 3, 1]),
 array([2, 2, 2, ..., 1, 0, 1]),
 array([2, 2, 2, ..., 3, 0, 3]),
 array([1, 1, 1, ..., 0, 1, 0]),
 array([2, 2, 2, ..., 2, 3, 1]),
 array([1, 1, 1, ..., 1, 1, 3]),
 array([1, 1, 1, ..., 0, 2, 2]),
 array([1, 1, 1, ..., 2, 0, 2]),
 array([1, 1, 1, ..., 2, 0, 3]),
 array([1, 1, 1, ..., 1, 0, 3]),
 array([1, 1, 1, ..., 1, 3, 2]),
 array([1, 1, 1, ..., 0, 1, 0]),
 array([1, 1, 1, ..., 2, 0, 0]),
 array([1, 1, 1, ..., 1, 2, 1]),
 array([1, 1, 1, ..., 3, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 2, 1])]

In [51]:
 # Fonction permettant de calculer des ARI par paires dans une liste de labels
def temp_stability_ari(labels_list):
  t=len(labels_list)
  # Nb d'individus du découpage initial
  nombre_lignes_cluster0 = labels_list[0].shape[0]
  # Calcul de l'ARI entre les paires de labels
  ari_score=[]
  for i in range(1,t):
      labels_list[i] = labels_list[i][:nombre_lignes_cluster0]
      ari = adjusted_rand_score(labels_list[0], labels_list[i])
      ari_score.append(ari)
      print(f"Le score de l'ARI entre les labels à la position 0 et ", i, "est de :", ari)
  return (ari_score)
  # Retourne la liste des labels des ARI calculés pour chaque paire de labels de la liste

In [52]:
ari_list=temp_stability_ari(label_list)
ari_list

Le score de l'ARI entre les labels à la position 0 et  1 est de : 0.9998453063611601
Le score de l'ARI entre les labels à la position 0 et  2 est de : 0.99977480498236
Le score de l'ARI entre les labels à la position 0 et  3 est de : 0.9997043076073553
Le score de l'ARI entre les labels à la position 0 et  4 est de : 0.9995306762516927
Le score de l'ARI entre les labels à la position 0 et  5 est de : 0.999609956318458
Le score de l'ARI entre les labels à la position 0 et  6 est de : 0.9994173873478409
Le score de l'ARI entre les labels à la position 0 et  7 est de : 0.9994689460546023
Le score de l'ARI entre les labels à la position 0 et  8 est de : 0.9616364107042696
Le score de l'ARI entre les labels à la position 0 et  9 est de : 0.961567644262383
Le score de l'ARI entre les labels à la position 0 et  10 est de : 0.9614301234131081
Le score de l'ARI entre les labels à la position 0 et  11 est de : 0.9614301234131081
Le score de l'ARI entre les labels à la position 0 et  12 est de : 

[0.9998453063611601,
 0.99977480498236,
 0.9997043076073553,
 0.9995306762516927,
 0.999609956318458,
 0.9994173873478409,
 0.9994689460546023,
 0.9616364107042696,
 0.961567644262383,
 0.9614301234131081,
 0.9614301234131081,
 0.9614502132367015,
 0.9612175857950294,
 0.9612175857950294,
 0.9611012931677974,
 0.9611700286021593,
 0.9611700286021593,
 0.9611700286021593,
 0.9611700286021593,
 0.9611700286021593]

A partir de la huitièmepériode, c'est à dire 8x15j, le score ARI descend légèrement.

Pour garder notre algorithme au maximum de son efficacité, nous pourrions proposer un contrat de maintenance tous les trois mois, soit 90 jours.